In [1]:
#!pip3 show torch
!nvidia-smi

Tue Nov  5 16:02:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-PCIE-16GB           On  | 00000000:21:00.0 Off |                    0 |
| N/A   30C    P0              24W / 250W |      4MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import pandas as pd
import numpy as np
import torch
import scripts
from functools import lru_cache
import torchmetrics
from torch import nn
import optuna

# Data loading

First we load the data. The basic idea is to create dictionaries with features associated to the drugs and cell-lines. In principle, the splits and the data shouldn't be changed

In [4]:
@lru_cache(maxsize = None)
def get_data(n_fold = 0, fp_radius = 2):
    # 1
    smile_dict = pd.read_csv("data/smiles.csv", index_col=0)
    fp = scripts.FingerprintFeaturizer(R = fp_radius)
    drug_dict = fp(smile_dict.iloc[:, 1], smile_dict.iloc[:, 0])
    # 2
    driver_genes = pd.read_csv("data/driver_genes.csv").loc[:, "symbol"].dropna()
    rnaseq = pd.read_csv("data/rnaseq_normcount.csv", index_col=0)
    driver_columns = rnaseq.columns.isin(driver_genes)
    filtered_rna = rnaseq.loc[:, driver_columns]
    tensor_exp = torch.Tensor(filtered_rna.to_numpy())
    cell_dict = {cell: tensor_exp[i] for i, cell in enumerate(filtered_rna.index.to_numpy())}
    # 3
    data = pd.read_csv("data/GDSC1.csv", index_col=0)
    # default, remove data where lines or drugs are missing:
    data = data.query("SANGER_MODEL_ID in @cell_dict.keys() & DRUG_ID in @drug_dict.keys()")
    unique_cell_lines = data.loc[:, "SANGER_MODEL_ID"].unique()
    # 4 
    np.random.seed(420) # for comparibility, don't change it!
    np.random.shuffle(unique_cell_lines)
    folds = np.array_split(unique_cell_lines, 10)
    test_lines = folds[0] # ?? 질문 필요. 0번째 fold를 test로 쓴다면, 이후에도 train_idx에서 0을 제거해야하는것 아닌가? 만약 train_idx에서 n_fold를 제거할 것이라면, trest_lines를 folds[n_fold]로 바꿔야하지 않는가? 
    train_idxs = list(range(10))
    train_idxs.remove(n_fold)
    np.random.seed(420)
    validation_idx = np.random.choice(train_idxs)
    train_idxs.remove(validation_idx)
    train_lines = np.concatenate([folds[idx] for idx in train_idxs])
    validation_lines = folds[validation_idx]
    test_lines = folds[n_fold] # 아 여기서 오버라이드 됐네? 그럼 위에 test_lines는 왜 있는거지?
    # 5
    train_data = data.query("SANGER_MODEL_ID in @train_lines")
    validation_data = data.query("SANGER_MODEL_ID in @validation_lines")
    test_data = data.query("SANGER_MODEL_ID in @test_lines")

    return (scripts.OmicsDataset(cell_dict, drug_dict, train_data),
    scripts.OmicsDataset(cell_dict, drug_dict, validation_data),
    scripts.OmicsDataset(cell_dict, drug_dict, test_data))

데이터를 처리하여 특정 cell line과 약물 간 상호작용을 예측하기 위한 데이터 셋을 구축한다.     
여기서는 데이터를 학습, 검증, 테스트용으로 분할하고 이를 `OmicsDataset`형태로 반환한다. 

1. 
   - `@lru_cache(maxsize=None)`는 `functools` 모듈에 포함된 데코레이터로 Least Recently Used 캐싱전략을 사용한다. 이 함수는 아래 함수의 결과를 캐시에 저장하여, 동일한 입력에 대해 함수 호출을 최적화한다. 최대 크기를 `None`으로 설정해 캐시가 무한으로 사용된다.
   - `fp`에서는 `FingerprintFeaturizer`를 사용하여 molecule fingerprint를 생성하는데, 이때 반경을 지정된 `fp_radius`로 지정하여 SMILES 데이터를 약물 딕셔너리(`drug_dict`)로 변환한다. 
2. 
   - `driver_genes` 유전자 심볼을 불러와 데이터에 결측치를 제거하고 리스트로 저장한다. 이후, `rnaseq`에 대한 데이터도 불러온다. `rnaseq.columns.isin(driver_genes)`는 `rnaseq` 의 각 칼럼의 이름이 `driver_genes` 목록에 포함되어 있는지를 `True` 혹은 `False`로 반환한다. 최종적으로 논리적 인덱스 배열을 생성하여 `driver_columns`에 저장한다. 이는 `True`인 유전자 열만 필터링하여 `filtered_rna`에 저장하는데 쓰인다.
   - 필터링된 유전자 컬럼은 `tensor`로 변환되어 `cell_dict`에 딕셔너리 형태로 저장된다. `Tensor`는 PyTorch에서 수치 데이터를 저장하고 다루기 위해 사용하는 기본적인 데이터 구조이다. 텐서는 GPU에서 연산이 가능하여 대규모 데이터에 대해 신경망을 이용해 빠른 계산을 수행할 수 있다. 
   - `filtered_rna`의 로우 인덱스(각 cell line의 이름 또는 ID)를 Numpy 배열로 변환하여, 각 cell line의 이름을 `cell`이라는 key, `tensor_exp[i]`를 RNA 발현값 텐서로 저장하여 딕셔너리로 만든다.   
3. 
   - GDSC 데이터를 불러와, `SANGER_MODEL_ID`가 `cell_dict`에, 그리고 `DRUG_ID`가 `drug_dict`에 존재하는 항목만 남긴다. 또한, 모든 행에서 `SANGER_MODEL_ID` 열만 선택하여, 중복을 제거하여 저장한다. 
4. 
   - 일괄적으로 `420`번 시드를 사용하여 `unique_cell_line`을 셔플하고, 데이터를 10개의 fold로 고정적으로 분할한다. 이때, 첫번째 fold를 `test_lines`로 사용한다. 
   - `train_idxs`에 0부터 9까지의 숫자로 구성된 리스트를 넣는다. 이는 이후 fold의 인덱스를 가리키게 된다. 이후 지정된 "n_fold"를 인덱스에서 삭제한다. 
   - train_idxs의 리스트에서 무작위로 하나의 인덱스를 선택하여 `validation_idx`로 사용한다. 
   - `folds[idx] for idx in train_idxs`는 train 데이터로 사용할 fold들의 세포주 ID 배열들을 추출하고, `np.concatenate`를 사용하여 추출된 배열들을 하나로 이어붙여 최종 train cell line ID list를 만든다. 
5. 
   - query는 데이터 프레임에서 SQL 스타일의 조건식을 통해 데이터를 필터링하는 메서드이다. 
   - 쿼리를 사용하여, 각 데이터셋에 `SANGER_MODEL_ID`가 포함된 row만 추출한다.

# Configuration

we declare the configuration, this is going to be model-specific and we get the datasets

In [5]:
config = {"features" : {"fp_radius":2}, # chemical의 fingerprint 생성 radius를 2로 설정
          "optimizer": {"batch_size": 220, # 한번에 학습시킬 데이터의 양
                        "clip_norm":19, # 그라디언트 클리핑에 사용할 최대 norm 값
                        "learning_rate": 0.0004592646200179472, # 학습률
                        "stopping_patience":15}, # 개선되지 않는 epoch가 15번 이상 나오면 학습을 중단
          "model":{"embed_dim":485, # input을 embedding할 때 사용할 차원
                 "hidden_dim":696, # hidden layer의 차원
                 "dropout":0.48541242824674574, # 40퍼센트의 노드를 랜덤하게 드랍아웃 
                 "n_layers": 4, # 3개의 hidden layer를 사용
                 "norm": "batchnorm"}, # batch normalization을 사용하여 모델이 학습 중 출력 분포를 정규화하여 학습을 안정화
         "env": {"fold": 0, # 0번째 fold를 사용하여 학습. 이는 음 n_fold에 들어갈 값을 의미하는 듯 하다. 
                "device":"cuda:2", # GPU자원을 사용할 장치를 지정한다. 
                 "max_epochs": 100, # 최대 epoch 수 
                 "search_hyperparameters":False}} # hyper parameter 이미 있으니 안쓴다.

# params={'batch_size': 220, 'clip_norm': 19, 'dropout': 0.48541242824674574, 'embed_dim': 485, 'hidden_dim': 696, 'learning_rate': 0.0004592646200179472, 'n_layers': 4, 'norm': 'batchnorm'}

- Gradient Clipping:
  -  Gradient의 크기를 threshold 이하로 제한하는 방식으로, 일반적으로 Norm을 기준으로 수행한다. 
  -  Norm Clipping은 Gradient vector의 Norm을 계산하고, 이 값이 지정된 threshold보다 크면 Gradient를 threshold로 scaling한다. 즉, 그냥 threshold 보다 큰값을 threshold 값으로 변환한다. 
  -  이는 Gradient가 지나치게 커지는 그라디엔트 폭주 현상을 막고, 모델이 안정적으로 학습할 수 있도록 돕는다. 
- Batch Normalization:
  - 인공 신경망 학습 시 각 레이어의 출력 분포를 정규화하여 학습 속도를 향상시키고, weight initialization에 대한 robustness를 증가시키고, 모델을 regularization한다. 
  - 딥러닝 모델이 매우 깊어질 때 발생하는 [Internal Covariate Shift](https://cvml.tistory.com/5)문제를 줄여준다. 
  - Covariant shift는 input data의 분포가 test와 train에서 각각 다르게 나타나는 현상을 의미한다. 그리고 이게 뉴럴 네트워크 내부에서 일어날 때 Internal covariant shift라고 부른다. 

In [6]:
train_dataset, validation_dataset, test_dataset = get_data(n_fold = config["env"]["fold"],
                                                           fp_radius = config["features"]["fp_radius"])

[16:02:21] DEPRECATION WARNING: please use MorganGenerator
[16:02:21] DEPRECATION WARNING: please use MorganGenerator
[16:02:21] DEPRECATION WARNING: please use MorganGenerator
[16:02:21] DEPRECATION WARNING: please use MorganGenerator
[16:02:21] DEPRECATION WARNING: please use MorganGenerator
[16:02:21] DEPRECATION WARNING: please use MorganGenerator
[16:02:21] DEPRECATION WARNING: please use MorganGenerator
[16:02:21] DEPRECATION WARNING: please use MorganGenerator
[16:02:21] DEPRECATION WARNING: please use MorganGenerator
[16:02:21] DEPRECATION WARNING: please use MorganGenerator
[16:02:21] DEPRECATION WARNING: please use MorganGenerator
[16:02:21] DEPRECATION WARNING: please use MorganGenerator
[16:02:21] DEPRECATION WARNING: please use MorganGenerator
[16:02:21] DEPRECATION WARNING: please use MorganGenerator
[16:02:21] DEPRECATION WARNING: please use MorganGenerator
[16:02:21] DEPRECATION WARNING: please use MorganGenerator
[16:02:21] DEPRECATION WARNING: please use MorganGenerat

# Hyperparameter optimization

we wrap the function for training the model in a function that can be used by optuna

In [7]:
def train_model_optuna(trial, config):
    # 1 
    def pruning_callback(epoch, train_r):
        trial.report(train_r, step = epoch)
        if np.isnan(train_r):
            raise optuna.TrialPruned()
        if trial.should_prune():
            raise optuna.TrialPruned()
    # 2
    config["model"] = {"embed_dim": trial.suggest_int("embed_dim", 64, 512),
                    "hidden_dim": trial.suggest_int("hidden_dim", 64, 2048),
                    "n_layers": trial.suggest_int("n_layers", 1, 6),
                    "norm": trial.suggest_categorical("norm", ["batchnorm", "layernorm", None]),
                    "dropout": trial.suggest_float("dropout", 0.0, 0.5)}
    config["optimizer"] = { "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-1, log=True),
                            "clip_norm": trial.suggest_int("clip_norm", 0.1, 20),
                            "batch_size": trial.suggest_int("batch_size", 128, 512),
                            "stopping_patience":10}
    # 3
    try:
        R, model = scripts.train_model(config,
                                       train_dataset,
                                       validation_dataset,
                                       use_momentum=True,
                                       callback_epoch = pruning_callback)
        return R
    except Exception as e:
        print(e)
        return 0

- Optuna 라이브러리를 사용하여 모델의 하이퍼파라미터를 자동으로 탐색하기위한 `train_model_optuna`함수를 정의
- Optuna는 베이지안 최적화 기반의 하이퍼파라미터 최적화 라이브러리이다. `trial`객체를 통해 하이퍼파라미터를 샘플링하고 모델을 학습시킨다. 
- 위에서 이미 설정한 기본 `config`의 일부 항목을 수정하여 테스트한다.  

1. 
    - `pruning_callback`함수를 정의한다. 이 함수는 학습 도중 성능이 저하되거나, 개선이 없을때 early stopping을 수행하는 콜백 함수(특정 이벤트 발생 시 자동 호출되는 함수)이다. pruning은 가지치기를 의미. 성능이 좋지않은 하이퍼파라미터 조합을 조기에 종료하여 속도 향상
    - `trial.report(train_r, step=epoch)`는 현재 epoch에서 학습성능(`train_r`)을 보고한다. 이를 통해 하이퍼파라미터의 성능을 평가한다.
    - `if np.isnan(train_r)`는 보고가 `NaN`인지 확인한다. `NaN`인 경우, 학습에 문제가 발생했음을 의미한다. 이때 `optuna.TrialPruned()` 예외를 발생시켜 학습을 조기에 종료한다. 
    - `if trial.should_prune()`는 Optuna가 특정 trial을 중단해야할 조건을 충족하면 예외를 발생시켜 조기종료한다. 
2. 
    - `trial.suggest_*` 메서드를 사용해 다양한 hyperparameter의 조합을 탐색. `trial` 객체는 무작위 샘플링 객체이다. 
    - 예를 들어 `("embed_dim", 64, 512)`경우에는, 64와 512사이의 값을 샘플링한다. 
    - `("norm", ["batchnorm", "layernorm", None])`에서는 셋 중 하나를 샘플링하여 레이어의 정규화 방식을 선택한다. 
3. 
    - `try`-`except`를 사용하여 오류를 처리한다. 
    - `scripts.train_model`을 호출하여 모델을 지정된 suggest된 하이퍼파라미터로 학습시키고, 최적화한 성능 지표 `R`과 학습된 모델을 반환한다. 
    - 오류가 발생할 경우 0을 반환하고 오류 내용을 출력한다. 이로 인해 모델이 특정 하이퍼파라미터 조합에서 학습되지 않으면, 해당 조합은 무시되고 최적화가 계속 진행된다. 

In [8]:
# 1
if config["env"]["search_hyperparameters"]:
    study_name = f"baseline_model"
    storage_name = "sqlite:///studies/{}.db".format(study_name)
    # 2
    study = optuna.create_study(study_name=study_name,
                                storage=storage_name,
                                direction='maximize',
                                load_if_exists=True,
                                pruner=optuna.pruners.MedianPruner(n_startup_trials=30,
                                                               n_warmup_steps=5,
                                                               interval_steps=5))
    # 3
    objective = lambda x: train_model_optuna(x, config)
    study.optimize(objective, n_trials=40)
    best_config = study.best_params
    print(best_config)
    config["model"]["embed_dim"] = best_config["embed_dim"]
    config["model"]["hidden_dim"] = best_config["hidden_dim"]
    config["model"]["n_layers"] = best_config["n_layers"]
    config["model"]["norm"] = best_config["norm"]
    config["model"]["dropout"] = best_config["dropout"]
    config["optimizer"]["learning_rate"] = best_config["learning_rate"]
    config["optimizer"]["clip_norm"] = best_config["clip_norm"]
    config["optimizer"]["batch_size"] = best_config["batch_size"]

1. 
    - "search_hyperparameters" 모드가 켜져있을때만 수행
2. 
    - Optuna에서 하이퍼파라미터 최적화를 수행할 때, `optuna.create_study()`와 `study.optimize()`가 실행된다.
    - 여기서 study name은 그저 baseline model이다. 하이퍼파라미터 탐색 결과는 SQLite 데이터베이스의 `studies/baseline_model.db`에 저장된다. 
    - `load_if_exists=True`는 동일한 `study_name`이 이미 존재하면 기존의 결과를 불러온다. 기존 결과를 이어서 탐색할 수 있게 해준다. 
    - `pruner=optuna.pruners.MedianPruner(...)`는 Pruning 전략으로 `MedianPruner`를 사용한다. 이는 일정 단계 후 현재 trial의 성능이 이전 trial 성능의 median보다 작을 경우 해당 trial을 중단시킨다. 
    - 여기서는 최소 30번의 trial, 첫 5번의 epoch(=step)를 Pruning 없이 실행하여 초기 성능을 확인한다. 이후 5 스텝마다 성능을 확인하고 Pruning 여부를 결정한다. Trial은 step을 포함한다. 
3. 
    - 미리 정의해둔 `train_model_optuna` 함수를 objective function으로 사용하고, 이걸 maximize한다. 
    - `study.optimize()`함수를 `n_trials`만큼 실행한다. 
    - best parameters를 받아 config을 업데이트한다. 

# Model training and evaluation

After we have a set of optimal hyperparameters we train our model. The train model function could be changed, but:
- test_dataset cannot be used until we call the final evaluation step
- the evaluation step cannot be modified, it must take the model produced by your pipeline, a dataloader that provides the correct data for your model, and the final metrics have to be printed

In [9]:
_, model = scripts.train_model(config, torch.utils.data.ConcatDataset([train_dataset, validation_dataset]), None, use_momentum=False)
device = torch.device(config["env"]["device"])
metrics = torchmetrics.MetricTracker(torchmetrics.MetricCollection(
    {"R_cellwise_residuals":scripts.GroupwiseMetric(metric=torchmetrics.functional.pearson_corrcoef,
                          grouping="drugs",
                          average="macro",
                          residualize=True),
    "R_cellwise":scripts.GroupwiseMetric(metric=torchmetrics.functional.pearson_corrcoef,
                          grouping="cell_lines",
                          average="macro",
                          residualize=False),
    "MSE":torchmetrics.MeanSquaredError()}))
metrics.to(device)
test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                       batch_size=config["optimizer"]["batch_size"],
                                       drop_last=False,
                                      shuffle=False)


/home/kim14/.local/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:43: FutureWarning: The default value for `maximize` will be changed from `True` to `None` in v1.7.0 of TorchMetrics,will automatically infer the value based on the `higher_is_better` attribute of the metric (if such attribute exists) or raise an error if it does not. If you are explicitly setting the `maximize` argument to either `True` or `False` already, you can ignore this warning.
  warnings.warn(*args, **kwargs)  # noqa: B028


epoch : 0: train loss: 1.9204309752527273 Smoothed R interaction (validation) None
epoch : 1: train loss: 1.5204692332249767 Smoothed R interaction (validation) None
epoch : 2: train loss: 1.4354846109196824 Smoothed R interaction (validation) None
epoch : 3: train loss: 1.3596970630141925 Smoothed R interaction (validation) None
epoch : 4: train loss: 1.3097563190842574 Smoothed R interaction (validation) None
epoch : 5: train loss: 1.2619699643467956 Smoothed R interaction (validation) None
epoch : 6: train loss: 1.2169348020036266 Smoothed R interaction (validation) None
epoch : 7: train loss: 1.1833453347098153 Smoothed R interaction (validation) None
epoch : 8: train loss: 1.1471325890073236 Smoothed R interaction (validation) None
epoch : 9: train loss: 1.124045192633035 Smoothed R interaction (validation) None
epoch : 10: train loss: 1.096354653925266 Smoothed R interaction (validation) None
epoch : 11: train loss: 1.0771197381446946 Smoothed R interaction (validation) None
epoc

- best parameters를 사용하여 모델을 트레이닝한다. 이 때, `train_dataset`과 `validation_dataset`을 `ConcatDataset`을 통해 하나로 병합하여 사용한다. 
- `use_momentom = False`를 통해 모멘텀을 사용하지 않는다. 
  - 모멘텀은 딥러닝 학습 시 최적화 알고리즘이 사용하는 기법 중 하나로, 이전 단계의 그라디언트를 일정 비율로 현재 단계에 반영하여 학습을 안정화하고 가속하는 역할을 한다. 
  - 이로 인해 좁은 골짜기 형태의 loss 곡면에서는 진동을 줄여주고, 완만한 경사면에서는 더 빠르게 이동하여 최적화 속도를 높인다. 그런데 여기서는 안쓴다고 했으니, 여기서는 이전 단계의 그라디언트 방향을 고려하지 않는다. 
- `env`의 `device` 항목(cuda:3)을 `torch.device`로 설정한다. 
- `MetricTracker`를 통해서 메트릭을 설정한다. `MetricCollection`을 통해 여러개의 메트릭을 정의한 후, 트래커로 묶어 관리한다. 
  - `R_cellwise_residual`에서 피어슨 상관계수를 기반으로, `drugs` 단위로 계산한다.  
  - 메트릭 계산 시 residual을 사용하여 성능을 평가하고, `average="macro"`는 그룹별 성능을 균등하게 반영하여 전체 평균을 계산한다.
  - `R_cellwise`는 `grouping="cell_lines"`로 세포주 단위로 계산한다. 얘는 residual 없이 메트릭을 계산한다.
- `metrics.to(device)`는 정의된 메트릭을 `device`에 전송하여 GPU에서 사용할 수 있게한다. 
- `torch.utils.data.DataLoader`를 통해, 테스트 데이터셋을 기반으로 데이터 로더를 생성하여, 모델의 성능을 테스트할 준비를 한다.
  - `drop_last=False`를 사용하여 마지막 배치의 샘플 수가 배치 크기보다 작을 경우에도 이를 버리지 않고 사용한다.

In [10]:
final_metrics = scripts.evaluate_step(model, test_dataloader, metrics, device)
print(final_metrics)

/work/kim14/project_work/scripts/models.py:66: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3683.)
  return torch.linalg.solve(A, Xy).T


{'MSE': 1.9241538047790527, 'R_cellwise': 0.8880401253700256, 'R_cellwise_residuals': 0.31074970960617065}
